In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [4]:
da = pd.read_csv('Davis Elec Hourly 1_24.2_25.csv')
da['Timestamp'] = pd.to_datetime(da['Timestamp'].str.extract(r'(.*?)(?= Los_Angeles)')[0])
da['Demand_kW'] = da.iloc[:,1].str.replace('kW', '').astype(float)
da['Usage_kWh'] = da.iloc[:,2].str.replace('kWh', '').astype(float)

sd = pd.read_csv('SaturdjaDai elec_hr 1_24-2_25.csv')
sd['Timestamp'] = pd.to_datetime(sd['Timestamp'].str.extract(r'(.*?)(?= Los_Angeles)')[0])
sd['Demand_kW'] = sd.iloc[:,1].str.replace('kW', '').astype(float)
sd['Usage_kWh'] = sd.iloc[:,2].str.replace('kWh', '').astype(float)

/var/folders/yl/_fjnz6h53yv8g0p40zrwgxsc0000gn/T/ipykernel_32419/2741872308.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  da['Timestamp'] = pd.to_datetime(da['Timestamp'].str.extract(r'(.*?)(?= Los_Angeles)')[0])
/var/folders/yl/_fjnz6h53yv8g0p40zrwgxsc0000gn/T/ipykernel_32419/2741872308.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  sd['Timestamp'] = pd.to_datetime(sd['Timestamp'].str.extract(r'(.*?)(?= Los_Angeles)')[0])


In [5]:
# https://www.pge.com/tariffs/assets/pdf/tariffbook/ELEC_SCHEDS_B-20.pdf

In [6]:
def repeat_dataset_for_years(df, years=15):
    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True).dt.tz_convert(None)

    repeated_dfs = []
    original_start = df['Timestamp'].min()

    for i in range(years):
        df_copy = df.copy()
        time_offset = pd.DateOffset(years=i)
        df_copy['Timestamp'] = df_copy['Timestamp'] + time_offset
        repeated_dfs.append(df_copy)

    return pd.concat(repeated_dfs, ignore_index=True)


In [7]:
def find_break_even_capacity_param(
    df, P=0.99, Bd=0.000027, Ci_per_kWh=300,
    start_E=7500, max_E=15000, step_size=500,
    csv_path='battery_cost_results.csv'
):
    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True).dt.tz_convert(None)
    df['month'] = df['Timestamp'].dt.month
    df['hour'] = df['Timestamp'].dt.hour

    # Precompute rate
    summer_months = [6, 7, 8, 9]
    super_off_peak_months = [3, 4, 5]
    super_off_peak_hours = range(9, 14)
    summer_peak = range(16, 21)
    summer_part_peak = list(range(14, 16)) + list(range(21, 23))
    winter_peak = range(16, 21)

    def get_rate(row):
        hour = row['hour']
        month = row['month']
        if month in summer_months:
            if hour in summer_peak: return 0.20832
            elif hour in summer_part_peak: return 0.16020
            else: return 0.12220
        else:
            if month in super_off_peak_months and hour in super_off_peak_hours: return 0.04451
            elif hour in winter_peak: return 0.17965
            else: return 0.12189

    df['rate'] = df.apply(get_rate, axis=1)
    df['GridOnlyCost'] = df['Usage_kWh'] * df['rate']
    grid_only_cost = df['GridOnlyCost'].sum()

    U = df['Usage_kWh'].values
    R = df['rate'].values
    T = len(U)
    df['date'] = df['Timestamp'].dt.date
    degradation_mask = (df['date'] != df['date'].shift(1)).astype(float).values

    # --- Define Variables and Parameter ---
    E_param = cp.Parameter(nonneg=True)
    B = cp.Variable(T)
    D = cp.Variable(T)
    C = cp.Variable(T)

    constraints = [B[0] == 0]
    for t in range(T):
        constraints += [
            B[t] >= 0,
            B[t] <= E_param,
            D[t] >= 0,
            D[t] <= P * B[t],
            C[t] >= 0,
            C[t] <= E_param
        ]
        if t > 0:
            constraints.append(B[t] == B[t-1] - D[t-1] / P + C[t-1])

    G = U - D + C / P
    constraints += [G >= 0]

    degradation_cost = Bd * cp.sum(cp.multiply(cp.multiply(degradation_mask, B), R))
    install_cost = Ci_per_kWh * cp.power(E_param, 0.75)
    battery_cost_expr = cp.sum(cp.multiply(G, R)) + degradation_cost + install_cost

    results = []
    for e in range(start_E, max_E + 1, step_size):
        E_param.value = e
        problem = cp.Problem(cp.Minimize(battery_cost_expr), constraints)
        problem.solve()
        if problem.status == 'optimal':
            results.append((e, battery_cost_expr.value))
            print(f"✅ E = {e} kWh, Total Cost = ${battery_cost_expr.value:.2f}")
        else:
            results.append((e, np.inf))
            print(f"⚠️ Solver failed at E = {e} kWh")

    # Convert to DataFrame and write to CSV
    results_df = pd.DataFrame(results, columns=["Battery_kWh", "Total_Cost"])
    results_df.to_csv(csv_path, index=False)
    print(f"\n📝 Results saved to: {csv_path}")

    # Find best result
    feasible_results = [(e, c) for e, c in results if np.isfinite(c)]
    if feasible_results:
        best_E, best_cost = min(feasible_results, key=lambda x: x[1])
        print(f"\n🎯 Best Battery Size: {best_E} kWh")
        print(f"🔋 Battery Total Cost: ${best_cost:.2f}")
        print(f"💰 Grid-Only Cost: ${grid_only_cost:.2f}")
        print(f"📉 Net Savings: ${grid_only_cost - best_cost:.2f}")
        return best_E, best_cost, grid_only_cost, results_df
    else:
        print("❌ No feasible solution found within the tested range.")
        return None, None, grid_only_cost, results_df

In [ ]:
find_break_even_capacity_param(repeat_dataset_for_years(sd, years=3))

/Users/joelt/S25/venv3.9/lib/python3.9/site-packages/cvxpy/reductions/solvers/solving_chain.py:241: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Disciplined Parametrized Programming, at https://www.cvxpy.org/tutorial/dpp/index.html
  warnings.warn(DPP_ERROR_MSG)
